In [ ]:
# ───────────────────────────────────────────────────────
# notebook_cell [0]  : 기본 경로 & PYTHONPATH
# ───────────────────────────────────────────────────────
import sys, os
from pathlib import Path

# 프로젝트 루트
PRJ = Path(r"D:\project\Kaggle\BYU\byu-motor").resolve()

# src 모듈 import
for p in [PRJ, PRJ / "src"]:
    if str(p) not in sys.path:
        sys.path.insert(0, str(p))

print("sys.path[0:2] =", sys.path[:2])


In [ ]:
# notebook_cell [1]
import copy
from src.configs.common_config import base_cfg   # 우리가 만든 common_config
from src.configs.cfg_motor_resnet34 import cfg as res34_cfg
from src.configs.cfg_motor_effb3   import cfg as effb3_cfg   # 필요 시

# ────────── CSV 경로 교체 ──────────
CSV_FOLDED = PRJ / "data" / "raw" / "train_labels_folds.csv"

# 학습 시 사용할 공통 옵션
COMMON = dict(
    train_df=str(CSV_FOLDED),
    val_df=None,                # fold 컬럼으로 자동 분리
    data_folder=str(PRJ / "data"),         # raw/processed/zarr 하위
    output_dir=str(PRJ / "outputs"),
    num_workers=8,
    pin_memory=True,
)

print("Fold CSV  :", CSV_FOLDED)


In [ ]:
# notebook_cell [2]  ───────── k-fold 학습 드라이버
from pathlib import Path
from types import SimpleNamespace
import numpy as np
import torch

from src.train import train          # 우리가 만든 train.py

N_FOLDS = 5
SEED0   = 42

# ─── 모델 · 학습 공통 설정 --------------------------------------------
COMMON = dict(
    # 필수·주요
    backbone      = "resnet34",
    arch          = "resnet34",
    in_channels   = 1,
    n_classes     = 1,
    roi_size      = (96, 96, 96),

    # loss weight → 리스트 권장 (warning 제거)
    class_weights = [256.0, 1.0],
    lvl_weights   = [1, 1],

    # 러너 하이퍼파라미터
    batch_size = 4,
    epochs     = 40,
    lr         = 5e-4,
    amp        = True,

    # 데이터 로딩 / 샘플링
    train_sub_epochs = 1,
    num_workers      = 12,
    prefetch_factor  = 4,
    train_sampler  = "oversample",   # 필요 시 명시
    pos_ratio      = 0.8,

# --- MixUp & Soft-Gaussian label -----------------------------
    mixup_p         = 0.3,
    mixup_beta      = 1.0,
    soft_sigma      = 4.0,            # σ(voxel) for gaussian label
)

DATA_ROOT = r"D:\project\Kaggle\BYU\byu-motor\data\/processed/zarr/train"
CSV_PATH  = r"D:\project\Kaggle\BYU\byu-motor\data\raw\train_labels_folds.csv"

fold_scores = []

# ─── K-fold 루프 -------------------------------------------------------
for f in range(N_FOLDS):
    print(f"\n▶ Fold {f}")
    cfg = SimpleNamespace(
        **COMMON,                     # ← 필수 필드 한꺼번에 주입
        data_root = DATA_ROOT,
        train_csv = CSV_PATH,
        seed      = SEED0 + f,
        fold      = f,
        output_dir= fr"D:\project\Kaggle\BYU\byu-motor\models\fold{f}",
    )

    Path(cfg.output_dir).mkdir(parents=True, exist_ok=True)

    best_fbeta = train(cfg)           # 수정한 train() 호출
    fold_scores.append(best_fbeta)
    print(f"✅ Fold {f} best Fβ: {best_fbeta:.3f}")

print(f"\n🔸 CV  mean Fβ: {np.mean(fold_scores):.3f}")


In [ ]:
# notebook_cell [3]
import numpy as np

print("\n===== 5-Fold 결과 =====")
for i, sc in enumerate(fold_scores):
    print(f"Fold {i}: {sc:.4f}")
print("-----------------------")
print(f"CV Mean: {np.mean(fold_scores):.4f}")
